In [ ]:
# Lab 3: 고급 검색 - 화재예방 가이드 (40분) - 실습 버전
# 소방청 민원처리 AI Agent 교육

"""
학습 목표:
- 다중 문서 소스 관리
- 소방기본법 + 화재예방 가이드 통합 검색
- 검색 결과 병합 및 활용
"""

# ============================================================
# 환경 설정
# ============================================================

import sys, os
if 'google.colab' in sys.modules:
    !pip install openai chromadb pdfplumber -q

import getpass
from openai import OpenAI
import pdfplumber
import chromadb
from chromadb.config import Settings
from typing import List, Dict

# API 키 설정
print("[API 키 입력] OpenAI API 키를 입력하세요")
api_key = getpass.getpass("API Key: ")
client = OpenAI(api_key=api_key)

print("[완료] 환경 설정 완료!\n")

# ============================================================
# PDF 파일 업로드
# ============================================================

from google.colab import files

print("[파일 업로드] 소방기본법.pdf (10페이지)와 화재예방.pdf (44페이지)를 업로드하세요")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"[완료] {filename} 업로드 완료")

# ============================================================
# Lab 2 함수 재사용 (이미 구현됨)
# ============================================================

def extract_text_from_pdf(pdf_path: str, max_pages: int = None) -> List[Dict]:
    """PDF에서 텍스트 추출"""
    pages_data = []
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages) if max_pages is None else min(max_pages, len(pdf.pages))
        for i in range(total_pages):
            text = pdf.pages[i].extract_text()
            pages_data.append({"page_num": i + 1, "text": text, "char_count": len(text)})
    return pages_data

def chunk_text(pages_data: List[Dict], chunk_size: int = 500, overlap: int = 50) -> List[Dict]:
    """텍스트 청킹"""
    chunks = []
    chunk_id = 0
    for page_data in pages_data:
        text = page_data['text']
        page_num = page_data['page_num']
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk_text = text[start:end]
            if chunk_text.strip():
                chunks.append({
                    "id": f"chunk_{chunk_id}",
                    "text": chunk_text,
                    "page": page_num,
                    "start_pos": start
                })
                chunk_id += 1
            start = end - overlap
    return chunks

def get_embedding(text: str) -> List[float]:
    """임베딩 생성"""
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

# ============================================================
# 실습 1: 소방기본법 Vector DB 재구축
# ============================================================

print("\n" + "="*60)
print("[실습 1] 소방기본법 Vector DB 재구축")
print("="*60)

# ChromaDB 초기화
print("\n[진행중] ChromaDB 초기화...")
chroma_client = chromadb.Client(Settings(
    persist_directory="./chroma_db",
    anonymized_telemetry=False
))

try:
    chroma_client.delete_collection("fire_law")
    chroma_client.delete_collection("fire_prevention")
except:
    pass

# 소방기본법 처리 (이미 구현됨 - 참고용)
print("\n[진행중] 소방기본법.pdf 처리...")
fire_law_pages = extract_text_from_pdf("소방기본법.pdf", max_pages=10)
fire_law_chunks = chunk_text(fire_law_pages)
print(f"[완료] {len(fire_law_chunks)}개 청크 생성")

fire_law_collection = chroma_client.create_collection("fire_law")

print("[진행중] 소방기본법 임베딩 및 저장...")
for i, chunk in enumerate(fire_law_chunks):
    if i % 10 == 0 and i > 0:
        print(f"  진행: {i}/{len(fire_law_chunks)}")
    embedding = get_embedding(chunk['text'])
    fire_law_collection.add(
        ids=[f"law_{chunk['id']}"],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{"page": chunk['page'], "source": "소방기본법"}]
    )

print(f"[완료] 소방기본법 {len(fire_law_chunks)}개 청크 저장 완료")

# ============================================================
# 실습 2: 화재예방 가이드 Vector DB 구축
# ============================================================

print("\n\n" + "="*60)
print("[실습 2] 화재예방 가이드 Vector DB 구축")
print("="*60)

"""
TODO: 화재예방 가이드도 소방기본법과 동일한 방식으로 처리하세요

작업 순서:
1. extract_text_from_pdf()로 화재예방.pdf 읽기
2. chunk_text()로 청킹
3. 컬렉션 생성 (이름: "fire_prevention")
4. 반복문으로 임베딩 및 저장
   - ids: f"prev_{chunk['id']}"
   - source: "화재예방가이드"

힌트:
- 소방기본법 처리 코드를 참고하되, 파일명과 컬렉션 이름만 변경
- prevention_pages, prevention_chunks 변수 사용
- prevention_collection 생성
"""

print("\n[진행중] 화재예방.pdf 처리...")
# TODO: 여기에 화재예방 PDF 처리 코드 작성

prevention_pages = []  # extract_text_from_pdf("화재예방.pdf")로 변경
prevention_chunks = []  # chunk_text(prevention_pages)로 변경

if prevention_chunks:
    print(f"[완료] {len(prevention_chunks)}개 청크 생성")
    
    # TODO: 컬렉션 생성
    prevention_collection = None  # chroma_client.create_collection(...)
    
    # TODO: 임베딩 및 저장 반복문 작성
    
    print(f"[완료] 화재예방 가이드 {len(prevention_chunks)}개 청크 저장 완료")
else:
    print("[아직 구현되지 않았습니다]")
    prevention_collection = None

# ============================================================
# 실습 3: 개별 검색 함수
# ============================================================

print("\n\n" + "="*60)
print("[실습 3] 개별 검색 함수 구현")
print("="*60)

def search_fire_law(query: str, n_results: int = 3) -> List[Dict]:
    """소방기본법 검색 (이미 구현됨 - 참고용)"""
    query_embedding = get_embedding(query)
    results = fire_law_collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    search_results = []
    for i in range(len(results['documents'][0])):
        search_results.append({
            "text": results['documents'][0][i],
            "page": results['metadatas'][0][i]['page'],
            "source": results['metadatas'][0][i]['source']
        })
    return search_results

def search_fire_prevention(query: str, n_results: int = 3) -> List[Dict]:
    """
    화재예방 가이드 검색
    
    TODO: search_fire_law와 동일한 구조로 구현하세요
    
    힌트:
    - prevention_collection.query() 사용
    - 나머지는 search_fire_law와 동일
    """
    
    if prevention_collection is None:
        return []
    
    # TODO: 여기에 검색 코드 작성
    # query_embedding = get_embedding(query)
    # results = prevention_collection.query(...)
    # return [...]
    
    return []

# 개별 검색 테스트
print("\n[테스트] 개별 검색")
test_query = "소화기 점검"

print(f"\n[검색어] {test_query}")
print("\n[소방기본법 검색 결과]")
law_results = search_fire_law(test_query, n_results=2)
for i, r in enumerate(law_results, 1):
    print(f"  {i}. {r['page']}페이지: {r['text'][:100]}...")

print("\n[화재예방 가이드 검색 결과]")
prev_results = search_fire_prevention(test_query, n_results=2)
if prev_results:
    for i, r in enumerate(prev_results, 1):
        print(f"  {i}. {r['page']}페이지: {r['text'][:100]}...")
else:
    print("[아직 구현되지 않았습니다]")

# ============================================================
# 실습 4: 통합 검색 시스템
# ============================================================

print("\n\n" + "="*60)
print("[실습 4] 통합 검색 시스템 - 두 소스 동시 검색")
print("="*60)

def search_both_sources(query: str, n_results_per_source: int = 2) -> Dict:
    """
    소방기본법 + 화재예방 가이드 동시 검색
    
    TODO: 두 검색 함수를 호출하여 결과를 통합하세요
    
    장점:
    1. 법령과 실무 가이드를 함께 참고 가능
    2. 더 풍부한 컨텍스트 제공
    3. 법적 근거 + 실무 방안 동시 제시
    
    힌트:
    - search_fire_law(query, n_results_per_source) 호출
    - search_fire_prevention(query, n_results_per_source) 호출
    - 딕셔너리로 반환: {"법령": [...], "예방가이드": [...], "total_count": ...}
    """
    
    # TODO: 여기에 통합 검색 코드 작성
    # law_results = search_fire_law(...)
    # prevention_results = search_fire_prevention(...)
    # return {"법령": law_results, "예방가이드": prevention_results, ...}
    
    return {
        "법령": [],
        "예방가이드": [],
        "total_count": 0
    }

# ============================================================
# 실습 5: 통합 검색 테스트
# ============================================================

print("\n[실습 5] 통합 검색 시스템 테스트")
print("="*60)

test_queries = [
    "소화기 관리 방법",
    "화재 발생 시 대응 절차",
    "비상구 설치 기준"
]

for query in test_queries:
    print(f"\n[검색어] {query}")
    print("-"*50)
    
    combined_results = search_both_sources(query, n_results_per_source=2)
    
    if combined_results['total_count'] > 0:
        print("\n[소방기본법 - 법적 근거]")
        for i, r in enumerate(combined_results['법령'], 1):
            print(f"  {i}. {r['page']}페이지")
            print(f"     {r['text'][:120]}...")
        
        print("\n[화재예방가이드 - 실무 가이드]")
        for i, r in enumerate(combined_results['예방가이드'], 1):
            print(f"  {i}. {r['page']}페이지")
            print(f"     {r['text'][:120]}...")
    else:
        print("[아직 구현되지 않았습니다]")
    
    print("\n" + "="*60)

# ============================================================
# 실습 6: 통합 답변 생성
# ============================================================

print("\n\n" + "="*60)
print("[실습 6] 통합 RAG 답변 생성")
print("="*60)

def answer_with_both_sources(question: str) -> str:
    """
    양쪽 소스를 활용한 답변 생성
    
    TODO: 통합 검색 결과를 컨텍스트로 LLM 답변 생성
    
    힌트:
    1. search_both_sources()로 검색
    2. 법령과 가이드 컨텍스트 각각 구성
    3. system 메시지:
       "소방기본법(법령)과 화재예방 가이드(실무)를 참고하여 답변"
       "1. 법적 근거 제시 2. 실무 방법 안내"
    4. user 메시지에 두 컨텍스트 모두 포함
    """
    
    # TODO: 여기에 통합 답변 생성 코드 작성
    
    return ""

# 테스트
print("\n[테스트] 통합 답변 생성")
test_question = "음식점 주방에서 화재 예방을 위해 어떤 조치가 필요한가요?"
print(f"\n[질문] {test_question}")
print("-"*50)
print("\n[답변]")
answer = answer_with_both_sources(test_question)
print(answer if answer else "[아직 구현되지 않았습니다]")

# ============================================================
# Lab 3 완료
# ============================================================

print("\n\n" + "="*60)
print("[완료] Lab 3 완료!")
print("="*60)
print("""
학습 내용:
1. 다중 문서 소스 관리 (2개 Vector DB)
2. 개별 검색 함수 구현
3. 통합 검색 시스템 구축
4. 법령 + 가이드 융합 답변 생성

핵심 함수:
- search_fire_law(): 소방기본법 검색
- search_fire_prevention(): 화재예방 검색
- search_both_sources(): 통합 검색 (Lab 4에서 사용)
- answer_with_both_sources(): 통합 답변

다음 Lab:
- Lab 4에서는 이 통합 검색을 활용하여 민원 처리 보고서를 자동 생성합니다.

실습 체크리스트:
[ ] 실습 2: 화재예방 가이드 Vector DB 구축
[ ] 실습 3: search_fire_prevention 함수 완성
[ ] 실습 4: search_both_sources 통합 검색 완성
[ ] 실습 6: answer_with_both_sources 답변 생성 완성
""")